In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.impute import SimpleImputer
from scipy.stats import chi2_contingency, pearsonr
import warnings
warnings.filterwarnings('ignore')

def detect_and_fix_data_leakage(df):
    """
    Systematically identify and remove data leakage, then perform clean analysis
    """
    
    print("=" * 80)
    print("DATA LEAKAGE DETECTION AND CLEAN METHODOLOGY")
    print("=" * 80)
    
    # ============================================================================
    # STEP 1: Identify All Potential Leakage Sources
    # ============================================================================
    print("\n" + "="*60)
    print("STEP 1: DATA LEAKAGE DETECTION")
    print("="*60)
    
    # Create educational target if not exists
    if 'completed_secondary' not in df.columns and 'v106' in df.columns:
        df['completed_secondary'] = (df['v106'] >= 2).astype(int)
    
    target_var = 'completed_secondary'
    
    if target_var not in df.columns:
        print("ERROR: Cannot create educational target variable")
        return None, None, None
    
    print(f"Target variable: {target_var}")
    print(f"Baseline rate: {df[target_var].mean()*100:.1f}%")
    
    # Identify potential leakage variables
    education_keywords = ['education', 'school', 'literacy', 'v106', 'v107', 'v149', 'v150', 'v151', 'v152']
    
    # Find suspicious variables
    suspicious_vars = []
    all_columns = df.columns.tolist()
    
    print("\nSUSPICIOUS VARIABLES (potential leakage):")
    print("-" * 45)
    
    for col in all_columns:
        col_lower = col.lower()
        if any(keyword in col_lower for keyword in education_keywords):
            suspicious_vars.append(col)
            
        # Check for perfect correlations
        if col != target_var and df[col].dtype in ['int64', 'float64']:
            try:
                valid_data = df[[col, target_var]].dropna()
                if len(valid_data) > 100:
                    corr = valid_data[col].corr(valid_data[target_var])
                    if abs(corr) > 0.9:  # Suspiciously high correlation
                        suspicious_vars.append(col)
                        print(f"{col:25} | Correlation: {corr:.4f} | POTENTIAL LEAKAGE")
            except:
                pass
    
    # Remove duplicates
    suspicious_vars = list(set(suspicious_vars))
    
    print(f"\nFound {len(suspicious_vars)} suspicious variables")
    for var in suspicious_vars:
        if var in df.columns:
            print(f"  - {var}")
    
    # ============================================================================
    # STEP 2: Create Clean Feature Set
    # ============================================================================
    print("\n" + "="*60)
    print("STEP 2: CLEAN FEATURE SELECTION")
    print("="*60)
    
    # Define safe, theoretically sound predictors
    safe_features = {
        # Basic demographics (available at birth/childhood)
        'v012': 'Current age',
        'v013': 'Age group', 
        'v101': 'Region',
        'v102': 'Urban/Rural residence',
        
        # Household characteristics
        'hv009': 'Household size',
        'v190': 'Wealth quintile',
        'v191': 'Wealth score',
        'hv270': 'Household wealth',
        'hv271': 'Household wealth score',
        
        # Cultural/social factors
        'v130': 'Religion',
        
        # Infrastructure access
        'hv206': 'Has electricity',
        'hv207': 'Has radio',
        'hv208': 'Has television',
        
        # Family structure
        'v501': 'Marital status',
        'v502': 'Marriage history'
    }
    
    # Check which safe features exist
    available_features = {}
    for var, description in safe_features.items():
        if var in df.columns:
            available_features[var] = description
    
    print("AVAILABLE SAFE FEATURES:")
    print("-" * 25)
    for var, desc in available_features.items():
        missing_pct = df[var].isna().mean() * 100
        print(f"{var:12} | {desc:25} | Missing: {missing_pct:5.1f}%")
    
    # ============================================================================
    # STEP 3: Clean Data Preparation
    # ============================================================================
    print("\n" + "="*60)
    print("STEP 3: CLEAN DATA PREPARATION")
    print("="*60)
    
    # Use only safe features
    feature_cols = list(available_features.keys())
    
    X = df[feature_cols].copy()
    y = df[target_var].copy()
    
    # Remove rows with missing target
    valid_mask = ~y.isna()
    X_clean = X[valid_mask].copy()
    y_clean = y[valid_mask].copy()
    
    print(f"Clean dataset shape: {X_clean.shape}")
    print(f"Features used: {len(feature_cols)}")
    print(f"Target distribution: {y_clean.value_counts().to_dict()}")
    
    # Handle missing values intelligently
    # Separate continuous and categorical variables
    continuous_vars = ['v012', 'v191', 'hv271', 'hv009']
    continuous_present = [var for var in continuous_vars if var in feature_cols]
    categorical_vars = [var for var in feature_cols if var not in continuous_present]
    
    X_processed = X_clean.copy()
    
    # Impute continuous with median
    if continuous_present:
        cont_imputer = SimpleImputer(strategy='median')
        X_processed[continuous_present] = cont_imputer.fit_transform(X_processed[continuous_present])
        print(f"Imputed {len(continuous_present)} continuous variables with median")
    
    # Impute categorical with mode  
    if categorical_vars:
        cat_imputer = SimpleImputer(strategy='most_frequent')
        X_processed[categorical_vars] = cat_imputer.fit_transform(X_processed[categorical_vars])
        print(f"Imputed {len(categorical_vars)} categorical variables with mode")
    
    # ============================================================================
    # STEP 4: Realistic Model Testing
    # ============================================================================
    print("\n" + "="*60)
    print("STEP 4: REALISTIC MODEL PERFORMANCE")
    print("="*60)
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(
        X_processed, y_clean, test_size=0.2, random_state=42, stratify=y_clean
    )
    
    # Test simpler models first
    models = {
        'Logistic Regression': LogisticRegression(random_state=42, class_weight='balanced', max_iter=1000),
        'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced', max_depth=10),
    }
    
    model_results = {}
    
    print("CLEAN MODEL RESULTS:")
    print("-" * 25)
    
    for name, model in models.items():
        # Fit model
        if name == 'Logistic Regression':
            # Scale features for logistic regression
            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)
            model.fit(X_train_scaled, y_train)
            y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
            
            # Cross validation with scaled data
            cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='roc_auc')
            
        else:
            # Tree-based model
            model.fit(X_train, y_train)
            y_pred_proba = model.predict_proba(X_test)[:, 1]
            
            # Cross validation
            cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc')
        
        # Calculate metrics
        test_auc = roc_auc_score(y_test, y_pred_proba)
        
        model_results[name] = {
            'test_auc': test_auc,
            'cv_mean': cv_scores.mean(),
            'cv_std': cv_scores.std()
        }
        
        print(f"{name:20} | AUC: {test_auc:.4f} | CV: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
    
    # Check if results are realistic (AUC should be 0.6-0.8 for social science)
    best_auc = max(result['test_auc'] for result in model_results.values())
    
    if best_auc > 0.95:
        print(f"\nWARNING: AUC {best_auc:.4f} still suspiciously high - may indicate remaining leakage")
    elif best_auc > 0.85:
        print(f"\nCAUTION: AUC {best_auc:.4f} is high - verify no leakage remains")
    else:
        print(f"\nSUCCESS: AUC {best_auc:.4f} appears realistic for educational prediction")
    
    # ============================================================================
    # STEP 5: Feature Importance Analysis (Clean)
    # ============================================================================
    print("\n" + "="*60)
    print("STEP 5: CLEAN FEATURE IMPORTANCE")
    print("="*60)
    
    # Use Random Forest for interpretability
    rf_clean = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
    rf_clean.fit(X_train, y_train)
    
    # Feature importance
    importance_df = pd.DataFrame({
        'feature': feature_cols,
        'importance': rf_clean.feature_importances_,
        'description': [available_features[f] for f in feature_cols]
    }).sort_values('importance', ascending=False)
    
    print("TOP PREDICTORS (clean analysis):")
    print("-" * 40)
    for idx, row in importance_df.head(10).iterrows():
        print(f"{row['feature']:12} | {row['description']:25} | Importance: {row['importance']:.4f}")
    
    # ============================================================================
    # STEP 6: Regional Analysis (Descriptive)
    # ============================================================================
    print("\n" + "="*60)
    print("STEP 6: REGIONAL EDUCATIONAL PATTERNS")
    print("="*60)
    
    if 'v101' in available_features:
        print("REGIONAL SECONDARY EDUCATION RATES:")
        print("-" * 35)
        
        region_names = {1: 'Kigali', 2: 'South', 3: 'West', 4: 'North', 5: 'East'}
        
        regional_stats = []
        for region_code, region_name in region_names.items():
            region_data = df[df['v101'] == region_code]
            if len(region_data) > 0:
                completion_rate = region_data[target_var].mean() * 100
                sample_size = len(region_data[target_var].dropna())
                
                regional_stats.append({
                    'region': region_name,
                    'rate': completion_rate,
                    'sample': sample_size
                })
                
                print(f"{region_name:8} | Rate: {completion_rate:5.1f}% | Sample: {sample_size:5,}")
        
        # Calculate disparities
        if regional_stats:
            rates = [stat['rate'] for stat in regional_stats]
            disparity = max(rates) - min(rates)
            print(f"\nRegional disparity: {disparity:.1f} percentage points")
    
    # ============================================================================
    # STEP 7: Wealth-Education Analysis
    # ============================================================================
    print("\n" + "="*60)
    print("STEP 7: WEALTH-EDUCATION GRADIENT")
    print("="*60)
    
    if 'v190' in available_features:
        print("EDUCATION RATES BY WEALTH QUINTILE:")
        print("-" * 35)
        
        wealth_labels = {1: 'Poorest', 2: 'Poorer', 3: 'Middle', 4: 'Richer', 5: 'Richest'}
        
        wealth_stats = []
        for quintile, label in wealth_labels.items():
            wealth_data = df[df['v190'] == quintile]
            if len(wealth_data) > 0:
                completion_rate = wealth_data[target_var].mean() * 100
                sample_size = len(wealth_data[target_var].dropna())
                
                wealth_stats.append({
                    'quintile': label,
                    'rate': completion_rate,
                    'sample': sample_size
                })
                
                print(f"{label:10} | Rate: {completion_rate:5.1f}% | Sample: {sample_size:5,}")
        
        # Calculate gradient
        if len(wealth_stats) >= 2:
            poorest_rate = wealth_stats[0]['rate']
            richest_rate = wealth_stats[-1]['rate']
            gradient = richest_rate - poorest_rate
            print(f"\nWealth gradient: {gradient:.1f} percentage points")
    
    # ============================================================================
    # STEP 8: Policy Recommendations
    # ============================================================================
    print("\n" + "="*60)
    print("STEP 8: EVIDENCE-BASED POLICY RECOMMENDATIONS")
    print("="*60)
    
    print("\nMETHODOLOGICAL FINDINGS:")
    print("-" * 25)
    print(f"• Clean prediction accuracy: {best_auc:.1%} (realistic for social science)")
    print(f"• Most important factors: {importance_df.head(3)['description'].tolist()}")
    
    if 'regional_stats' in locals() and regional_stats:
        lowest_region = min(regional_stats, key=lambda x: x['rate'])
        highest_region = max(regional_stats, key=lambda x: x['rate'])
        print(f"• Highest need region: {lowest_region['region']} ({lowest_region['rate']:.1f}%)")
        print(f"• Best performing region: {highest_region['region']} ({highest_region['rate']:.1f}%)")
    
    if 'wealth_stats' in locals() and len(wealth_stats) >= 2:
        print(f"• Wealth inequality: {gradient:.1f} percentage point gap between richest and poorest")
    
    print("\nPOLICY IMPLICATIONS:")
    print("-" * 20)
    print("• Geographic targeting needed for regional disparities")
    print("• Wealth-based interventions critical for equity")
    print("• Predictive models can identify at-risk populations")
    print("• Infrastructure access (electricity, media) impacts educational outcomes")
    
    return model_results, importance_df, available_features

# Execute the clean analysis
if __name__ == "__main__":
    dataset_path = r"C:\Users\USER\Desktop\MUKABUGINGO_THESIS_CODES\ANALYSIS\rwanda_dhs_processed.csv"
    
    try:
        df = pd.read_csv(dataset_path)
        print(f"Dataset loaded: {df.shape}")
        
        # Run clean analysis
        results, importance, features = detect_and_fix_data_leakage(df)
        
        if results is not None:
            print("\n" + "="*80)
            print("CLEAN ANALYSIS COMPLETED SUCCESSFULLY")
            print("Data leakage eliminated - results are now methodologically sound")
            print("="*80)
        
    except FileNotFoundError:
        print("Dataset not found. Please check the file path.")
    except Exception as e:
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()

Dataset loaded: (14634, 66)
DATA LEAKAGE DETECTION AND CLEAN METHODOLOGY

STEP 1: DATA LEAKAGE DETECTION
Target variable: completed_secondary
Baseline rate: 32.7%

SUSPICIOUS VARIABLES (potential leakage):
---------------------------------------------
has_secondary_plus        | Correlation: 1.0000 | POTENTIAL LEAKAGE

Found 12 suspicious variables
  - has_education
  - age_education_interaction
  - v107_scaled
  - v106
  - urban_education_interaction
  - v150
  - v152
  - has_secondary_plus
  - v149
  - education_wealth_interaction
  - v151
  - v107

STEP 2: CLEAN FEATURE SELECTION
AVAILABLE SAFE FEATURES:
-------------------------
v012         | Current age               | Missing:   0.0%
v013         | Age group                 | Missing:   0.0%
v101         | Region                    | Missing:   0.0%
v102         | Urban/Rural residence     | Missing:   0.0%
hv009        | Household size            | Missing:   0.0%
v190         | Wealth quintile           | Missing:   0.0%
v191 